# Packed Ensemble Application to the AirfRANS dataset

## Colab setup

Set `colab` to `True` if you wish to use it 

In [ ]:
colab = False
if colab:
    from google.colab import drive

    drive.mount("/content/drive")
    !source/content/drive/MyDrive/my_colab_env/bin/activate
    import sys
    import os

    sys.path.append("/content/drive/MyDrive/my_colab_env/lib/python3.10/site-packages")
    os.chdir("/content/drive/MyDrive/ml4science/ml4physim_startingkit")

    sys.path.append(os.getcwd())

## Installation

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS)

In [ ]:
# !pip install -r requirements.txt
# or
# !pip install -U .


Install the AirfRANS package

In [ ]:
# !pip install airfrans

## Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import torch
import torch.nn.functional as F

from lips import get_root_path
from lips.dataset.scaler.standard_scaler import StandardScaler
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import download_data

from my_packed_ensemble import *
from my_packed_cv import *

## Generic Step (Load the required data) <a id='generic_step'></a>

In [ ]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = '../Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [ ]:
BENCH_CONFIG_PATH = os.path.join("..", "airfoilConfigurations", "benchmarks",
                                 "confAirfoil.ini")  #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join("airfoilConfigurations", "simulators", "torch_fc.ini")  #Configuration file re

Download the data

In [ ]:
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

Loading the dataset using the dedicated class used by LIPS platform offers a list of advantages:

1. Ease the importing of datasets
1. A set of functions to organize the `inputs` and `outputs` required by augmented simulators


In [ ]:
# Load the required benchmark datasets, if it is not pickled, pickle it for future use
try:
    with open('benchmark.pkl', 'rb') as f:
        benchmark = pickle.load(f)
except:
    benchmark = AirfRANSBenchmark(benchmark_path=DIRECTORY_NAME,
                                  config_path=BENCH_CONFIG_PATH,
                                  benchmark_name=BENCHMARK_NAME,
                                  log_path=LOG_PATH)
    benchmark.load(path=DIRECTORY_NAME)
    with open('benchmark.pkl', 'wb') as f:
        pickle.dump(benchmark, f)

In [ ]:
dict = {
    "evaluateonly": False,
      "scoringonly": False,
      "simulator_config": {
        "simulator_type": "custom",
        "simulator_file": "my_packed_ensemble_mk2",
        "name": "MyAugmentedSimulator",
        "model": "AugmentedSimulator",
        "scaler_type": "None"
       },
      "simulator_extra_parameters": {
        "input_size": 7,
        "output_size": 4,
        "hidden_sizes": [64,64,8,64,64,64,8,64,64],
        "M": 8,
        "alpha": 4,
        "gamma": 1,
        "device": "cuda",
    
        "batch_size": 1,
        "nb_epochs": 600,
        "lr": 0.001,
        "subsampling": 32000
      },
      "training_config": {}
}

In [ ]:
import new_submission.my_packed_ensemble_mk2 as my_packed_ensemble_mk2

simulator = my_packed_ensemble_mk2.AugmentedSimulator(benchmark, **dict["simulator_extra_parameters"])

In [ ]:
simulator.train(benchmark.train_dataset)

## Model selection (Cross validation)

Importing the necessary dependencies, as well as the `packed_ensemble` methods

Create cross validation on hyperparameters of the model defined by ``param_grid``

In [ ]:
param_grid = {
    'hidden_sizes': [(48, 128, 48), (128, 256, 128)],
    'dropout': [True, False],
    "alpha": [2, 4],
    "gamma": [2, 4],
    "M": [4],
    'lr': [1e-2, 1e-3]
}

The `param_grid` will be divided in 3 partitions, each one will be executed on a different machine.

- Anton - partition 0
- Anthony - partition 1
- Alexi - partition 2

Change it in the cell below.

In [ ]:
partition = 1
device="cuda" if torch.cuda.is_available() else "cpu"

# hyperparameter tuning using CV
hyperparameters_tuning(benchmark=benchmark, param_grid=param_grid, k_folds=4, num_epochs=100, batch_size=500000, shuffle=True, n_workers=6,
                        scaler=StandardScaler(), partition=partition, verbose=True, size_scale=0.3, device=device)

<br></br>

---

## Model training

Define input and output sizes of the model

In [ ]:
input_size, output_size = infer_input_output_size(benchmark.train_dataset)

create a Packed MLP model

In [ ]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = PackedMLP(input_size=input_size,
                  output_size=output_size,
                  hidden_sizes=(50, 100, 50),
                  activation=F.relu,
                  device=device,
                  dropout=True,
                  )
model.to(device)
print(model.device)
print(model)

Create the `trainloader`

In [ ]:
train_loader = model.process_dataset(benchmark.train_dataset, training=True, n_workers=6)

Training the model

In [ ]:
model, train_losses, _ = train(model, train_loader, epochs=1, device=device, lr=3e-4)

##### prediction on `test_dataset`
This dataset has the same distribution as the training set

In [ ]:
predictions, observations = predict(model, benchmark._test_dataset, device=device)

In [ ]:
print("Prediction dimensions: ", predictions["x-velocity"].shape, predictions["y-velocity"].shape,
      predictions["pressure"].shape, predictions["turbulent_viscosity"].shape)
print("Observation dimensions:", observations["x-velocity"].shape, observations["y-velocity"].shape,
      observations["pressure"].shape, observations["turbulent_viscosity"].shape)
print("We have good dimensions!")

In [ ]:
from lips.evaluation.airfrans_evaluation import AirfRANSEvaluation

evaluator = AirfRANSEvaluation(config_path=BENCH_CONFIG_PATH,
                               scenario=BENCHMARK_NAME,
                               data_path=DIRECTORY_NAME,
                               log_path=LOG_PATH)

observation_metadata = benchmark._test_dataset.extra_data
metrics = evaluator.evaluate(observations=observations,
                             predictions=predictions,
                             observation_metadata=observation_metadata)
print(metrics)

##### Prediction on `test_ood_dataset`
This dataset has a different distribution in comparison to the training set.

In [ ]:
predictions, observations = predict(model, benchmark._test_ood_dataset, device=device)
evaluator = AirfRANSEvaluation(config_path=BENCH_CONFIG_PATH,
                               scenario=BENCHMARK_NAME,
                               data_path=DIRECTORY_NAME,
                               log_path=LOG_PATH)

metrics = evaluator.evaluate(observations=observations,
                             predictions=predictions,
                             observation_metadata=observation_metadata)
print(metrics)